# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with

mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
db_list = mongo.list_database_names()
print(db_list)

if 'uk_food' in db_list:
    print("'uk_food' database created correctly.")
else:
    print("uk_food database does not exist.")

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruit_DB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']
'uk_food' database created correctly.


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db.list_collection_names()


['establishments']

In [6]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 551803,
 'LocalAuthorityBusinessID': 'PI/000070948',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6586c408ffaa9e545f78d306'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/551803',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate':

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
restaurant_dict = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(restaurant_dict)


In [10]:
# Check that the new restaurant was inserted
establishments.find_one({'BusinessName': "Penang Flavours"})

{'_id': ObjectId('6586e6f105d242c001918cb9'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
select_fields = ['BusinessTypeID', 'BusinessType']

# Create a dictionary to return selected fields
fields = dict([(field, 1) for field in select_fields])

# Do not return '_id' field
fields['_id'] = 0

# Find the BusinessTypeID
getID = establishments.find_one({'BusinessType': "Restaurant/Cafe/Canteen"}, fields)

# Print the result
print(getID)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
query = {'$set': {'BusinessTypeID': getID['BusinessTypeID']}}
establishments.update_one({'BusinessName': "Penang Flavours"}, query)

In [13]:
# Confirm that the new restaurant was updated
select_fields = ['BusinessName', 'BusinessTypeID']
fields = dict([(field, 1) for field in select_fields])

establishments.find_one({'BusinessName': "Penang Flavours"}, fields)

{'_id': ObjectId('6586e6f105d242c001918cb9'),
 'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName': "Dover"})

994

In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': "Dover"})

In [16]:
# Check if any remaining documents include Dover
establishments.count_documents({'LocalAuthorityName': "Dover"})

0

In [17]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('6586c408ffaa9e545f78d5ed'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
convert_long = {'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}}}
convert_lat = {'$set': {'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}

establishments.update_many({}, [convert_long, convert_lat])

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [20]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}])

In [21]:
# Check that the coordinates and rating value are now numbers
select_fields = ['geocode', 'RatingValue']
fields = dict([(field, 1) for field in select_fields])

# Return a query with the columns to check
check_query = establishments.find_one({}, fields)

# Loop through the dictionary
for key, value in check_query.items():
    if isinstance(value, dict):
        # Value is an embedded dictionary, drill down to get the values
        for coord_key, coord_val in value.items():
            print(f'Field: {coord_key}\nType: {type(coord_val)}\n')
    else:
        print(f'Field: {key}\nType: {type(value)}\n')

Field: _id
Type: <class 'bson.objectid.ObjectId'>

Field: RatingValue
Type: <class 'int'>

Field: longitude
Type: <class 'float'>

Field: latitude
Type: <class 'float'>

